In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from pathlib import Path
from functools import reduce

In [8]:
path_folder='../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/features'
def read_pa(path_folder):
    files= list(Path(path_folder).glob('*.parquet'))
    for f in files:
        df=pd.read_parquet(f)
        print (df.head(2))
    return
read_pa(path_folder)

   index  bytes
0      0  6.245
1      1  3.985
   index visitor_type  is_commercial  is_unknown  visitor_type_freq  \
0      0      unknown              0           1           0.253658   
1      1      network              0           0           0.086453   

  visitor_country  country_freq  is_top_country  
0   other_country      0.855195               0  
1   other_country      0.855195               0  
   index  response response_class
0      0       200     successful
1      1       200     successful
   index           timestamp  hour  day  weekday  month  is_weekend
0      0 1995-07-01 00:00:01     0    1        5      7           1
1      1 1995-07-01 00:00:06     0    1        5      7           1
   index reqmethod         reqdirectory reqresourcetype  reqpathdepth  \
0      0       GET     /history/apollo/       Directory             3   
1      1       GET  /shuttle/countdown/       Directory             3   

   is_dynamic  requrllength  
0           0            16  
1 

In [ ]:
path_folder='../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/features'
def join_columns(path_folder,mode: str="merge",key: str="index",drop_index_col: bool = False,how: str="outer"):
    files= list(Path(path_folder).glob('*.parquet'))

    dfs =[]
    for f in files:
        df=pd.read_parquet(f)
        if mode == "concat":
            # Nếu cột "index" chỉ là cột thừa do save file thì bỏ
            if drop_index_col and key in df.columns:
                df = df.drop(columns=[key])
            dfs.append(df)

        elif mode == "merge":
            if key not in df.columns:
                raise ValueError(f"File {f.name} không có cột key '{key}'")

            dfs.append(df)

        else:
            raise ValueError("mode chỉ nhận 'concat' hoặc 'merge'")

    if mode == "concat":
        out = pd.concat(dfs, ignore_index=True)
    else:
        out = reduce(lambda a, b: pd.merge(a, b, on=key, how=how), dfs)
    return out
df=join_columns(path_folder)


In [41]:
df.head(5)

,index,bytes,visitor_type,is_commercial,is_unknown,visitor_type_freq,visitor_country,country_freq,is_top_country,response,...,day,weekday,month,is_weekend,reqmethod,reqdirectory,reqresourcetype,reqpathdepth,is_dynamic,requrllength
0,0,6.245,unknown,0,1,0.253658,other_country,0.855195,0,200,...,1,5,7,1,GET,/history/apollo/,Directory,3,0,16
1,1,3.985,network,0,0,0.086453,other_country,0.855195,0,200,...,1,5,7,1,GET,/shuttle/countdown/,Directory,3,0,19
2,2,4.085,unknown,0,1,0.253658,other_country,0.855195,0,200,...,1,5,7,1,GET,/shuttle/missions/sts-73/,Page,4,0,44
3,3,0.000,commercial,1,0,0.299574,other_country,0.855195,0,304,...,1,5,7,1,GET,/shuttle/countdown/,Page,3,0,31
4,4,4.179,unknown,0,1,0.253658,other_country,0.855195,0,200,...,1,5,7,1,GET,/shuttle/missions/sts-73/,Image,4,0,47


In [11]:
df.isnull().sum()

index                    0
bytes                28490
visitor_type             0
is_commercial            0
is_unknown               0
visitor_type_freq        0
visitor_country          0
country_freq             0
is_top_country           0
response                 0
response_class           0
timestamp                0
hour                     0
day                      0
weekday                  0
month                    0
is_weekend               0
reqmethod                0
reqdirectory             0
reqresourcetype          0
reqpathdepth             0
is_dynamic               0
requrllength             0
dtype: int64

In [43]:
df.dtypes

index                         int64
bytes                       float64
visitor_type                 object
is_commercial                 int64
is_unknown                    int64
visitor_type_freq           float64
visitor_country              object
country_freq                float64
is_top_country                int64
response                      int16
response_class               object
timestamp            datetime64[us]
hour                          int32
day                           int32
weekday                       int32
month                         int32
is_weekend                    int64
reqmethod                    object
reqdirectory                 object
reqresourcetype              object
reqpathdepth                  int64
is_dynamic                    int64
requrllength                  int64
dtype: object

In [12]:
df["response_class"].value_counts()

response_class
successful      2642411
redirection      275274
client error      17150
server error         95
Name: count, dtype: int64

In [15]:
df[df['bytes'].isnull()]['response'].value_counts()

response
404    17028
302    11174
200      136
403      111
501       31
400       10
Name: count, dtype: int64

In [21]:
def add_bytes_imputed(df):
    """fill NaN"""
    df = df.copy()
    df["bytes"] = pd.to_numeric(df["bytes"], errors="coerce")
    df["bytes_imputed"] = df["bytes"]

    df.loc[(df["response_class"]=="redirection") & df["bytes_imputed"].isna(), "bytes_imputed"] = 0

    med = df.groupby("response")["bytes"].median()
    m5 = (df["response_class"]=="server error") & df["bytes_imputed"].isna()
    df.loc[m5, "bytes_imputed"] = df.loc[m5, "response"].map(med).fillna(0)
    return df
df=add_bytes_imputed(df)


In [20]:
def flag_missing_bytes(df):
    df["bytes_missing"] = df["bytes"].isna().astype("int8")
    return df
df=flag_missing_bytes(df)
df.head(5)


,index,bytes,visitor_type,is_commercial,is_unknown,visitor_type_freq,visitor_country,country_freq,is_top_country,response,...,month,is_weekend,reqmethod,reqdirectory,reqresourcetype,reqpathdepth,is_dynamic,requrllength,bytes_imputed,bytes_missing
0,0,6.245,unknown,0,1,0.253658,other_country,0.855195,0,200,...,7,1,GET,/history/apollo/,Directory,3,0,16,6.245,0
1,1,3.985,network,0,0,0.086453,other_country,0.855195,0,200,...,7,1,GET,/shuttle/countdown/,Directory,3,0,19,3.985,0
2,2,4.085,unknown,0,1,0.253658,other_country,0.855195,0,200,...,7,1,GET,/shuttle/missions/sts-73/,Page,4,0,44,4.085,0
3,3,0.000,commercial,1,0,0.299574,other_country,0.855195,0,304,...,7,1,GET,/shuttle/countdown/,Page,3,0,31,0.000,0
4,4,4.179,unknown,0,1,0.253658,other_country,0.855195,0,200,...,7,1,GET,/shuttle/missions/sts-73/,Image,4,0,47,4.179,0


In [30]:
df.columns

Index(['index', 'bytes', 'visitor_type', 'is_commercial', 'is_unknown',
       'visitor_type_freq', 'visitor_country', 'country_freq',
       'is_top_country', 'response', 'response_class', 'timestamp', 'hour',
       'day', 'weekday', 'month', 'is_weekend', 'reqmethod', 'reqdirectory',
       'reqresourcetype', 'reqpathdepth', 'is_dynamic', 'requrllength',
       'bytes_imputed', 'bytes_missing'],
      dtype='str')

In [40]:
df['bytes_missing'].value_counts()

bytes_missing
0    2906440
1      28490
Name: count, dtype: int64

In [49]:
df['bytes_imputed'].isnull().sum()

np.int64(17285)

In [ ]:
def _entropy(s: pd.Series) -> float:
    s = s.dropna()
    if len(s) == 0:
        return 0.0
    p = s.value_counts(normalize=True)
    return float(-(p * np.log2(p + 1e-12)).sum())

def _top_share(s: pd.Series) -> float:
    s = s.dropna()
    if len(s) == 0:
        return 0.0
    return float(s.value_counts().iloc[0] / len(s))

def make_train_table(df: pd.DataFrame, freq: str) -> pd.DataFrame:
    d = df.copy()
    d["timestamp"] = pd.to_datetime(d["timestamp"], errors="coerce")
    d = d.dropna(subset=["timestamp"]).sort_values("timestamp").set_index("timestamp")

    #đảm bảo numeric
    d["bytes"] = pd.to_numeric(d["bytes"], errors="coerce")
    d["bytes_imputed"] = pd.to_numeric(d["bytes_imputed"], errors="coerce")
    d["response"] = pd.to_numeric(d["response"], errors="coerce")

    r=d.resample(freq)

    out = pd.DataFrame(index=r.size().index)

    # Labels 
    out["y_req"] = r["bytes"].size().astype("int64")
    out["y_bytes_imp"] = r["bytes_imputed"].sum()


    # Bytes quality 
    out["bytes_missing_rate"] = r["bytes_missing"].mean().fillna(0.0)
    out["bytes_all_missing"] = (out["bytes_missing_rate"] == 1.0).astype("int8")

    # Health / response mix 
    out["error_rate"] = (d["response"] >= 400).resample(freq).mean().fillna(0.0)
    out["server_error_rate"] = (d["response"] >= 500).resample(freq).mean().fillna(0.0)
    out["redirection_rate"] = ((d["response"] >= 300) & (d["response"] < 400)).resample(freq).mean().fillna(0.0)

    # Mix rates 
    out["dynamic_rate"] = r["is_dynamic"].mean().fillna(0.0)
    out["commercial_rate"] = r["is_commercial"].mean().fillna(0.0)
    out["unknown_rate"] = r["is_unknown"].mean().fillna(0.0)

    # URL/path 
    out["avg_url_len"] = r["requrllength"].mean()
    out["avg_path_depth"] = r["reqpathdepth"].mean()

    # Diversity / concentration 
    out["country_nunique"] = r["visitor_country"].nunique()
    out["dir_nunique"] = r["reqdirectory"].nunique()
    out["method_nunique"] = r["reqmethod"].nunique()

    out["top_country_share"] = r["visitor_country"].apply(_top_share).fillna(0.0)
    out["endpoint_entropy"] = r["reqdirectory"].apply(_entropy).fillna(0.0)

    #  Time features 
    out = out.reset_index()
    out["hour"] = out["timestamp"].dt.hour
    out["weekday"] = out["timestamp"].dt.weekday
    out["is_weekend"] = (out["weekday"] >= 5).astype("int8")

    return out

def make_1m_5m_15m_train(df: pd.DataFrame):
    return {
        "1m": make_train_table(df, "1min"),
        "5m": make_train_table(df, "5min"),
        "15m": make_train_table(df, "15min"),
    }


In [53]:
tables = make_1m_5m_15m_train(df)
df_1m, df_5m, df_15m = tables["1m"], tables["5m"], tables["15m"]

In [54]:
display(df_1m)
display(df_5m)
display(df_15m)

,timestamp,y_req,y_bytes_imp,bytes_missing_rate,bytes_all_missing,error_rate,server_error_rate,redirection_rate,dynamic_rate,commercial_rate,...,avg_url_len,avg_path_depth,country_nunique,dir_nunique,method_nunique,top_country_share,endpoint_entropy,hour,weekday,is_weekend
0,1995-07-01 00:00:00,42,608.453,0.000000,0,0.000000,0.0,0.071429,0.000000,0.261905,...,29.523810,2.904762,3,13,1,0.928571,2.884403,0,5,1
1,1995-07-01 00:01:00,61,910.128,0.016393,0,0.016393,0.0,0.049180,0.016393,0.442623,...,31.901639,3.032787,2,18,1,0.934426,3.411756,0,5,1
2,1995-07-01 00:02:00,57,628.556,0.035088,0,0.000000,0.0,0.192982,0.000000,0.385965,...,28.736842,3.035088,3,19,1,0.929825,3.715645,0,5,1
3,1995-07-01 00:03:00,71,1747.389,0.014085,0,0.000000,0.0,0.169014,0.000000,0.704225,...,32.070423,3.000000,2,21,1,0.985915,3.601396,0,5,1
4,1995-07-01 00:04:00,70,1383.149,0.000000,0,0.000000,0.0,0.057143,0.000000,0.542857,...,32.400000,2.885714,2,17,1,0.985714,3.079143,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76315,1995-08-22 23:55:00,13,319.726,0.000000,0,0.000000,0.0,0.000000,0.000000,0.615385,...,31.153846,2.615385,3,4,1,0.692308,1.546594,23,1,0
76316,1995-08-22 23:56:00,21,436.101,0.000000,0,0.000000,0.0,0.000000,0.000000,0.285714,...,31.190476,2.809524,3,10,1,0.809524,2.944395,23,1,0
76317,1995-08-22 23:57:00,29,334.910,0.000000,0,0.000000,0.0,0.137931,0.000000,0.551724,...,30.551724,3.000000,2,12,1,0.931034,3.147304,23,1,0
76318,1995-08-22 23:58:00,29,283.098,0.448276,0,0.448276,0.0,0.000000,0.000000,0.620690,...,45.448276,3.655172,3,12,1,0.793103,2.891790,23,1,0


,timestamp,y_req,y_bytes_imp,bytes_missing_rate,bytes_all_missing,error_rate,server_error_rate,redirection_rate,dynamic_rate,commercial_rate,...,avg_url_len,avg_path_depth,country_nunique,dir_nunique,method_nunique,top_country_share,endpoint_entropy,hour,weekday,is_weekend
0,1995-07-01 00:00:00,301,5277.675,0.013289,0,0.003322,0.0,0.109635,0.003322,0.491694,...,31.126246,2.973422,4,35,1,0.956811,3.866601,0,5,1
1,1995-07-01 00:05:00,267,5041.043,0.011236,0,0.000000,0.0,0.101124,0.007491,0.490637,...,31.501873,3.022472,4,38,1,0.883895,3.900038,0,5,1
2,1995-07-01 00:10:00,242,6111.846,0.033058,0,0.016529,0.0,0.165289,0.008264,0.388430,...,30.235537,3.107438,5,38,1,0.892562,3.999979,0,5,1
3,1995-07-01 00:15:00,282,4559.748,0.024823,0,0.014184,0.0,0.109929,0.017730,0.411348,...,31.063830,3.177305,4,49,2,0.904255,4.426225,0,5,1
4,1995-07-01 00:20:00,319,7262.385,0.006270,0,0.000000,0.0,0.075235,0.003135,0.592476,...,31.451411,3.131661,4,38,1,0.965517,3.977305,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15259,1995-08-22 23:35:00,127,2410.781,0.007874,0,0.000000,0.0,0.157480,0.015748,0.299213,...,30.488189,2.858268,4,35,1,0.661417,4.095580,23,1,0
15260,1995-08-22 23:40:00,119,3383.983,0.008403,0,0.000000,0.0,0.067227,0.016807,0.268908,...,27.352941,2.638655,3,25,1,0.739496,3.575257,23,1,0
15261,1995-08-22 23:45:00,128,1618.155,0.000000,0,0.000000,0.0,0.031250,0.031250,0.367188,...,32.617188,3.031250,4,33,1,0.835938,4.084003,23,1,0
15262,1995-08-22 23:50:00,137,1169.948,0.014599,0,0.014599,0.0,0.043796,0.014599,0.540146,...,29.532847,2.890511,2,33,1,0.912409,3.945751,23,1,0


,timestamp,y_req,y_bytes_imp,bytes_missing_rate,bytes_all_missing,error_rate,server_error_rate,redirection_rate,dynamic_rate,commercial_rate,...,avg_url_len,avg_path_depth,country_nunique,dir_nunique,method_nunique,top_country_share,endpoint_entropy,hour,weekday,is_weekend
0,1995-07-01 00:00:00,810,16430.564,0.018519,0,0.006173,0.0,0.123457,0.006173,0.460494,...,30.983951,3.029630,5,60,1,0.913580,4.102642,0,5,1
1,1995-07-01 00:15:00,945,18046.060,0.013757,0,0.005291,0.0,0.097354,0.007407,0.522751,...,31.714286,3.156614,7,69,2,0.923810,4.317121,0,5,1
2,1995-07-01 00:30:00,1006,22420.824,0.006958,0,0.003976,0.0,0.094433,0.006958,0.545726,...,33.388668,3.260437,7,63,1,0.885686,3.963939,0,5,1
3,1995-07-01 00:45:00,804,24088.038,0.013682,0,0.012438,0.0,0.077114,0.012438,0.534826,...,33.274876,3.259950,7,61,2,0.889303,4.222621,0,5,1
4,1995-07-01 01:00:00,750,17178.646,0.008000,0,0.002667,0.0,0.124000,0.020000,0.489333,...,32.257333,3.109333,6,64,1,0.886667,4.290682,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,1995-08-22 22:45:00,435,7308.498,0.002299,0,0.002299,0.0,0.110345,0.000000,0.340230,...,31.494253,2.981609,6,56,2,0.885057,4.057854,22,1,0
5084,1995-08-22 23:00:00,510,5480.335,0.009804,0,0.007843,0.0,0.125490,0.005882,0.282353,...,30.854902,3.009804,4,73,1,0.884314,4.665178,23,1,0
5085,1995-08-22 23:15:00,393,6420.996,0.000000,0,0.000000,0.0,0.058524,0.010178,0.422392,...,30.381679,2.944020,6,49,2,0.806616,4.155377,23,1,0
5086,1995-08-22 23:30:00,336,6792.788,0.011905,0,0.005952,0.0,0.107143,0.011905,0.324405,...,29.404762,2.770833,5,52,1,0.717262,4.090367,23,1,0


In [61]:
df_1m.isnull().sum()

timestamp                0
y_req                    0
y_bytes_imp              0
bytes_missing_rate       0
bytes_all_missing        0
error_rate               0
server_error_rate        0
redirection_rate         0
dynamic_rate             0
commercial_rate          0
unknown_rate             0
avg_url_len           7852
avg_path_depth        7852
country_nunique          0
dir_nunique              0
method_nunique           0
top_country_share        0
endpoint_entropy         0
hour                     0
weekday                  0
is_weekend               0
flag_data_gap            0
dtype: int64

In [60]:
def flag_data_gap(df):
    df["flag_data_gap"]=df["avg_url_len"].isna().astype('int8')
    return df
df_1m=flag_data_gap(df_1m)
df_5m=flag_data_gap(df_5m)
df_15m=flag_data_gap(df_15m)
df_1m.columns

Index(['timestamp', 'y_req', 'y_bytes_imp', 'bytes_missing_rate',
       'bytes_all_missing', 'error_rate', 'server_error_rate',
       'redirection_rate', 'dynamic_rate', 'commercial_rate', 'unknown_rate',
       'avg_url_len', 'avg_path_depth', 'country_nunique', 'dir_nunique',
       'method_nunique', 'top_country_share', 'endpoint_entropy', 'hour',
       'weekday', 'is_weekend', 'flag_data_gap'],
      dtype='str')

In [69]:
df_1m.to_parquet("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/model_ml/train_1m.parquet")
df_5m.to_parquet("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/model_ml/train_5m.parquet")
df_15m.to_parquet("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/model_ml/train_15m.parquet")